This Jupyter Notebook appears to focus on quantum chemistry simulations using Qiskit Nature and related libraries. Here's a summary of the tasks being performed:

1. **Setup and Initialization**:
    - Install and import necessary libraries for quantum chemistry and quantum computing.
    - Define the molecular system (LiH molecule) and its properties, such as geometry, charge, and spin multiplicity.

2. **Electronic Structure Problem**:
    - Use the PySCF driver to compute the electronic structure of the molecule.
    - Extract information like the number of spatial orbitals, spin configurations, and reference energy.

3. **Quantum Algorithms**:
    - Solve the electronic structure problem using classical and quantum algorithms, such as:
      - Hartree-Fock (HF) method.
      - Ground-state energy calculation using the NumPyMinimumEigensolver and Jordan-Wigner Mapper.
      - Variational Quantum Eigensolver (VQE) and UCCSD ansatz.

4. **State Preparation and Evolution**:
    - Prepare quantum states (e.g., Hartree-Fock state) and simulate their time evolution using Pauli evolution gates.
    - Use simulators like `AerSimulator` and `StatevectorSampler` to compute state probabilities and analyze results.

5. **Spin Operators and Observables**:
    - Compute spin operators (e.g., \( S_x, S_y, S_z, S^2 \)) and map them to qubit operators.
    - Evaluate expectation values of these operators for specific quantum states.

6. **Matrix Construction and Eigenvalue Problems**:
    - Construct Hamiltonian and overlap matrices for selected states.
    - Solve generalized eigenvalue problems to compute energies and eigenstates.

7. **Full Configuration Interaction (FCI)**:
    - Perform FCI calculations using PySCF to obtain highly accurate ground and excited state energies for comparison.

8. **Analysis and Visualization**:
    - Analyze results, such as state probabilities, normalized states, and expectation values.
    - Visualize quantum states and their properties.

This notebook combines classical and quantum computational techniques to study the electronic structure and dynamics of the LiH molecule, providing insights into its quantum properties.


In [18]:
import qiskit
import qiskit_aer
import qiskit_nature
import qiskit_nature_pyscf
import qiskit_algorithms
import qiskit_nature_pyscf
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.circuit.library import EfficientSU2
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_aer import  AerSimulator
# from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicBasis
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from scipy.linalg import eigh
from qiskit_nature.second_q.circuit.library import HartreeFock
from scipy.linalg import expm
from qiskit.quantum_info import Statevector
from numpy import pi
import pylatexenc
import numpy as np
import matplotlib.pyplot as plt


In [19]:
print(qiskit.__version__)
print(qiskit_aer.__version__)
print(qiskit_algorithms.__version__)
print(qiskit_nature.__version__)
print(qiskit_nature_pyscf.__version__)

1.4.0
0.17.0
0.3.1
0.7.2
0.4.0


# Analysis of LiH in 12 spin orbitals:

In [20]:
# Define the Li molecule
molecule = MoleculeInfo(
    symbols=["Li", "H"],
    coords=([0.0, 0.0, 0.0], [0.0, 0.0, 1.59]),  # Approximate bond distance for He2
    multiplicity=1,  # Singlet state
    charge=0
)

# Set up the PySCF driver
mdriver = PySCFDriver.from_molecule(
    molecule=molecule,
    basis="sto3g"
)


# You can now use the driver to run a calculation
electronic_structure_problem = mdriver.run()

In [21]:
print("Spatial orbitals:", electronic_structure_problem.num_spatial_orbitals)
print("Number of up spin: ", electronic_structure_problem.num_alpha)
print("Number of down spin: ", electronic_structure_problem.num_beta)
print("\n")
print("Down spin configuration : ", electronic_structure_problem.orbital_occupations_b)
print("Down spin configuration : ",electronic_structure_problem.orbital_occupations)
print("Number of particles: ", electronic_structure_problem.num_particles)
# print(electronic_structure_problem.second_q_ops()[0])

Spatial orbitals: 6
Number of up spin:  2
Number of down spin:  2


Down spin configuration :  [1. 1. 0. 0. 0. 0.]
Down spin configuration :  [1. 1. 0. 0. 0. 0.]
Number of particles:  (2, 2)


In [22]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

result = solver.solve(electronic_structure_problem)
print(result)
print("HF energy : ", electronic_structure_problem.reference_energy)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.880919855331
  - computed part:      -8.880919855331
~ Nuclear repulsion energy (Hartree): 0.998447567774
> Total ground state energy (Hartree): -7.882472287557
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  3.00466454]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  4.82267435296]
    - computed part:      [0.0  0.0  4.82267435296]
  > Dipole moment (a.u.): [0.0  0.0  -1.81800981296]  Total: 1.81800981296
                 (debye): [0.0  0.0  -4.620919590112]  Total: 4.620919590112
 
HF energy :  -7.862174819763764


In [23]:
mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(electronic_structure_problem.second_q_ops()[0])
state = HartreeFock(electronic_structure_problem.num_spatial_orbitals, electronic_structure_problem.num_particles, mapper)

In [ ]:
# state.draw(output="mpl", filename="HF.png")
#-7.876631325621035)

## QASM Simulator

In [25]:
# from qiskit.primitives import StatevectorSampler
# from qiskit import QuantumCircuit
# from qiskit.circuit.library import PauliEvolutionGate
# from qiskit_aer import Aer
# from qiskit import transpile

# evolution_times = np.linspace(0.0, 10, 10)  # List of time evolution values
# evolution_times = 1.0 * 3.1415* evolution_times # Time parameter for the unitary evolution
# shots = 1000000


# mapper = JordanWignerMapper()
# hf_state = HartreeFock(
#     electronic_structure_problem.num_spatial_orbitals,
#     electronic_structure_problem.num_particles,
#     mapper
# )

# # Step 3: Backend setup
# backend = Aer.get_backend("qasm_simulator")

# # # Initialize Sampler
# # sampler = StatevectorSampler()
# shots = 1000000

# # Step 4: Loop over different time evolutions
# state_probabilities = {}

# for t in evolution_times:
#     # Prepare circuit
#     qc = QuantumCircuit(hf_state.num_qubits)
#     qc.compose(hf_state, inplace=True)  # HF reference state
#     evolution_op = PauliEvolutionGate(qubit_hamiltonian, time=t)
#     qc.append(evolution_op, range(hf_state.num_qubits))  # Time evolution
#     qc.measure_all()

#     # Transpile and simulate
#     tqc = transpile(qc, backend)
#     result = backend.run(tqc, shots=shots).result()
#     counts = result.get_counts()
#     # job = sampler.run([qc], shots=shots)
#     # result = job.result()[0]
#     # counts = result.data['meas'].get_counts()

#     # Normalize counts to probabilities
#     # total_shots = sum(counts.values())
#     probabilities = {state: count / shots for state, count in counts.items()}
#     print(f"time {t} : ",probabilities)
#     # Store probabilities for this time
#     state_probabilities[t] = probabilities


## StateVector Sampler

In [26]:
from qiskit.primitives import StatevectorSampler
from qiskit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit_aer import Aer
from qiskit import transpile

evolution_times = np.linspace(0, 0.5, 10)  # List of time evolution values
evolution_times = 1.0 * 3.1415* evolution_times # Time parameter for the unitary evolution



mapper = JordanWignerMapper()
hf_state = HartreeFock(
    electronic_structure_problem.num_spatial_orbitals,
    electronic_structure_problem.num_particles,
    mapper
)

# # Step 3: Backend setup
# backend = Aer.get_backend("qasm_simulator")

# # Initialize Sampler
sampler = StatevectorSampler()
shots = 100000

# Step 4: Loop over different time evolutions
state_probabilities = {}

for t in evolution_times:
    # Prepare circuit
    qc = QuantumCircuit(hf_state.num_qubits)
    qc.compose(hf_state, inplace=True)  # HF reference state
    evolution_op = PauliEvolutionGate(qubit_hamiltonian, time=t)
    qc.append(evolution_op, range(hf_state.num_qubits))  # Time evolution
    qc.measure_all()

    # Transpile and simulate
    # tqc = transpile(qc, backend)
    # result = backend.run(tqc, shots=shots).result()
    # counts = result.get_counts()
    job = sampler.run([qc], shots=shots)
    result = job.result()[0]
    counts = result.data['meas'].get_counts()

    # Normalize counts to probabilities
    # total_shots = sum(counts.values())
    probabilities = {state: count / shots for state, count in counts.items()}
    print(f"time {t} : ",probabilities)
    # Store probabilities for this time
    state_probabilities[t] = probabilities


time 0.0 :  {'000011000011': 1.0}
time 0.17452777777777778 :  {'000011000011': 0.99927, '100001100001': 0.00057, '100001000101': 5e-05, '000101100001': 5e-05, '001001001001': 1e-05, '010001010010': 1e-05, '010001010001': 1e-05, '000110000110': 2e-05, '001010001010': 1e-05}
time 0.34905555555555556 :  {'000011000011': 0.99734, '100001100001': 0.00186, '000101100001': 0.00019, '010001010001': 0.00011, '100001000101': 0.00017, '000101000101': 1e-05, '001001001001': 9e-05, '100001000011': 4e-05, '000110000110': 5e-05, '100010100010': 2e-05, '000101000011': 2e-05, '000110000011': 4e-05, '010010010001': 2e-05, '001001001010': 1e-05, '001010001010': 2e-05, '000011000101': 1e-05}
time 0.5235833333333333 :  {'000011000011': 0.99411, '100001100001': 0.00418, '000110000110': 0.00013, '100001000101': 0.00027, '000101100001': 0.00028, '010010010010': 1e-05, '010001010001': 0.00028, '000011100001': 3e-05, '010001010010': 2e-05, '001001001001': 0.00015, '000011000101': 5e-05, '000110000011': 6e-05, '

In [27]:
# write python code to write state_probabilities to a text file
with open("state_probabilities.txt", "w") as f:
    for t, probs in state_probabilities.items():
        f.write(f"Time: {t}\n")
        for state, prob in probs.items():
            f.write(f"{state}: {prob}\n")
        f.write("\n")

In [28]:
from pyscf import gto, scf
from qiskit_nature.second_q.properties.s_operators import s_minus_operator, s_plus_operator, s_x_operator, s_y_operator, s_z_operator
# Define a molecule (e.g., LiH)
mol = gto.Mole()
mol.atom = """
Li 0.0 0.0 0.0
H  0.0 0.0 1.59
"""
mol.basis = "sto3g"
mol.build()

# Compute the overlap matrix in the atomic orbital (AO) basis
ao_overlap_matrix = mol.intor("int1e_ovlp")
print("AO Overlap Matrix:")
print(ao_overlap_matrix)
# Perform a Hartree-Fock calculation to get MO coefficients

hf = scf.RHF(mol)
hf.kernel()
mo_coeff = hf.mo_coeff

# Transform AO overlap matrix to MO basis
mo_overlap_matrix = mo_coeff.T @ ao_overlap_matrix @ mo_coeff

print("MO Overlap Matrix:")
print(mo_overlap_matrix)

AO Overlap Matrix:
[[1.         0.24113665 0.         0.         0.         0.06795133]
 [0.24113665 1.         0.         0.         0.         0.39776959]
 [0.         0.         1.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.        ]
 [0.         0.         0.         0.         1.         0.51477359]
 [0.06795133 0.39776959 0.         0.         0.51477359 1.        ]]
converged SCF energy = -7.86217481976376
MO Overlap Matrix:
[[ 1.00000000e+00 -7.69025335e-18 -3.03298238e-18  6.83794746e-19
   3.07273557e-19 -3.05096015e-18]
 [-9.22189899e-18  1.00000000e+00  5.42855131e-17  7.02871960e-17
   3.43637763e-18 -1.54449140e-17]
 [ 1.11757428e-17  1.45023348e-16  1.00000000e+00  2.91350037e-19
  -1.96784342e-17 -1.54712210e-16]
 [ 6.83794746e-19  7.02871960e-17  2.91350037e-19  1.00000000e+00
   8.27795499e-17  1.65749540e-16]
 [ 3.07273557e-19  3.43637763e-18 -1.96784342e-17  1.87444800e-16
   1.00000000e+00 -5.74876794e-17]
 

In [29]:
# overlap = ao_overlap_matrix
overlap = None
overlap = mo_overlap_matrix
num_spatial_orbitals=electronic_structure_problem.num_spatial_orbitals

In [30]:
s_x = s_x_operator(num_spatial_orbitals, overlap=overlap)
s_y = s_y_operator(num_spatial_orbitals, overlap=overlap)
s_z = s_z_operator(num_spatial_orbitals)
s_squared = (s_x @ s_x) + (s_y @ s_y) + (s_z @ s_z)

In [31]:
s_squared = mapper.map(s_squared)

In [32]:
# Define the Li molecule
molecule = MoleculeInfo(
    symbols=["Li", "H"],
    coords=([0.0, 0.0, 0.0], [0.0, 0.0, 1.59]),  # Approximate bond distance for He2
    multiplicity=1,  # Singlet state
    charge=0
)

# Set up the PySCF driver
mdriver = PySCFDriver.from_molecule(
    molecule=molecule,
    basis="sto3g"
)


# You can now use the driver to run a calculation
electronic_structure_problem = mdriver.run()
# print(electronic_structure_problem.second_q_ops()[0])

In [33]:
s_minus_operator(num_spatial_orbitals, overlap=overlap)

FermionicOp({'+_6 -_0': (1.0000000000000004+0j), '+_7 -_1': (1.0000000000000002+0j), '+_8 -_2': (1.0000000000000004+0j), '+_9 -_3': (1.0000000000000004+0j), '+_10 -_4': (1+0j), '+_11 -_5': (1.000000000000001+0j)}, num_spin_orbitals=12, )

In [34]:
s_plus_operator(num_spatial_orbitals, overlap=overlap)

FermionicOp({'+_0 -_6': (1.0000000000000004+0j), '+_1 -_7': (1.0000000000000002+0j), '+_2 -_8': (1.0000000000000004+0j), '+_3 -_9': (1.0000000000000004+0j), '+_4 -_10': (1+0j), '+_5 -_11': (1.000000000000001+0j)}, num_spin_orbitals=12, )

In [35]:
s_squared

SparsePauliOp(['XZZZZYXZZZZY', 'YZZZZYYZZZZY', 'XZZZZXYZZZZY', 'YZZZZXXZZZZY', 'XZZZZYYZZZZX', 'YZZZZYXZZZZX', 'XZZZZXXZZZZX', 'YZZZZXYZZZZX', 'IIIIIIIIIIII', 'IIIIIZIIIIIZ', 'IIXZZYIIXZZY', 'IIYZZYIIYZZY', 'IIXZZXIIYZZY', 'IIYZZXIIXZZY', 'IIXZZYIIYZZX', 'IIYZZYIIXZZX', 'IIXZZXIIXZZX', 'IIYZZXIIYZZX', 'IIIIXYIIIIXY', 'IIIIYYIIIIYY', 'IIIIXXIIIIYY', 'IIIIYXIIIIXY', 'IIIIXYIIIIYX', 'IIIIYYIIIIXX', 'IIIIXXIIIIXX', 'IIIIYXIIIIYX', 'IXZZZYIXZZZY', 'IYZZZYIYZZZY', 'IXZZZXIYZZZY', 'IYZZZXIXZZZY', 'IXZZZYIYZZZX', 'IYZZZYIXZZZX', 'IXZZZXIXZZZX', 'IYZZZXIYZZZX', 'IIIXZYIIIXZY', 'IIIYZYIIIYZY', 'IIIXZXIIIYZY', 'IIIYZXIIIXZY', 'IIIXZYIIIYZX', 'IIIYZYIIIXZX', 'IIIXZXIIIXZX', 'IIIYZXIIIYZX', 'XZZZYIXZZZYI', 'YZZZYIYZZZYI', 'XZZZXIYZZZYI', 'YZZZXIXZZZYI', 'XZZZYIYZZZXI', 'YZZZYIXZZZXI', 'XZZZXIXZZZXI', 'YZZZXIYZZZXI', 'IIXZYIIIXZYI', 'IIYZYIIIYZYI', 'IIXZXIIIYZYI', 'IIYZXIIIXZYI', 'IIXZYIIIYZXI', 'IIYZYIIIXZXI', 'IIXZXIIIXZXI', 'IIYZXIIIYZXI', 'IIIIZIIIIIZI', 'IXZZYIIXZZYI', 'IYZZYIIYZZYI', 'IXZZXIIY

In [36]:
state = Statevector.from_label("100001100001")

In [37]:
print(state.expectation_value(s_squared))

(-4.440892098500626e-16+0j)
